### import

In [ ]:
from google.colab import drive
from google.colab import files
import shutil
import os
import zipfile
import gdown
import albumentations as A
from albumentations.core.transforms_interface import ImageOnlyTransform
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### 마운트 시도 (실패)


> memo Gdown으로 했더니 에러가 나는 거 같고, 마운트를 하면 내 드라이브에서 하라고 하고, 공유에서는 안 된다고 하고.. 무슨 API 서비스 사용하라고 하는데 그거는 하기까지 시간이 오래 걸릴 거 같고...



In [ ]:
# drive.mount('/content/drive')

# drive_folder = "/content/drive/MyDrive/original_image/original_image"
# zip_filename = "bread.zip"

# os.chdir("/content")

# with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
#     for root, dirs, files in os.walk(drive_folder):
#         for file in files:
#             file_path = os.path.join(root, file)
#             zipf.write(file_path, os.path.relpath(file_path, drive_folder))

# files.download(zip_filename)

# print(f"데이터셋 다운 / 압해 완료")

# # https://drive.google.com/drive/folders/1TEHUAt9HuD3rH5iqrdJQAB70cmOXcn_9?usp=sharing

### 정석님이 올려두신 labeling data + image set (새로운 라벨링 위해서 주석 처리)

In [ ]:
# # 정석님이 올려두신 roboflow labeling data 가져오기

# # dataset_path = "/content/bread/original_image" # 원본 이미지 경로

# label_zip_file_id = "10VZf9IqZ0bVHFCIcELrWmLS01Kh0pzF-"
# labeling_output_name = "image_label_data.zip"
# # https://drive.google.com/file/d/10VZf9IqZ0bVHFCIcELrWmLS01Kh0pzF-/view?usp=drive_link

# # 기존 파일 있으면 삭제
# if os.path.exists(labeling_output_name):
#     os.remove(labeling_output_name)
#     print(f"기존 파일 삭제")

# gdown.download(f"https://drive.google.com/uc?id={label_zip_file_id}", labeling_output_name, quiet=False)
# print(f"labeling zip 다운")

# # 압축해제 절대경로 설정 했지만 안되는 거 같아서 그냥 두기로
# labels_path = "/content/bread/"

# # 기존 폴더 있으면 삭제
# if os.path.exists(labels_path):
#     shutil.rmtree(labels_path)
#     print(f"기존 폴더 삭제")

# os.makedirs(labels_path, exist_ok=True)

# # 압해
# label_zip_file_path = os.path.join(os.getcwd(), labeling_output_name)

# with zipfile.ZipFile(label_zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(labels_path)

# print(f"압축 해제 데이터 경로: {labels_path}")

Downloading...
From (original): https://drive.google.com/uc?id=10VZf9IqZ0bVHFCIcELrWmLS01Kh0pzF-
From (redirected): https://drive.google.com/uc?id=10VZf9IqZ0bVHFCIcELrWmLS01Kh0pzF-&confirm=t&uuid=1ba515fe-b0fa-4b66-83d4-e6eeb56a8f90
To: /content/image_label_data.zip
100%|██████████| 53.4M/53.4M [00:02<00:00, 23.1MB/s]


labeling zip 다운
압축 해제 데이터 경로: /content/bread/


# ⚠️윤곽선 포함 사진으로 라벨링 -> 훈련 과정 새로 시작


> 정석님이 올려주신 labeling 데이터를 사용하려고 했으나, 증강 이미지가 포함된 걸로 다시 라벨링이 필요하다는 것을 알게 됐음. edge/contours_detection 적용한 사진 파일을 저장하고, 빵 폴더별로 구분해서 포함시켜놓은 다음에, 다시 라벨링하고, gdown하고, 훈련 돌리면 된다.



### gdown(이전 버전)(주석처리)

In [ ]:
# zip_file_id = "1kSApTuj4bC3vtHJ-0t2HTkPaay7CTt0H"
# output_name = "bread_data.zip"

# # 기존 파일 있으면 삭제
# if os.path.exists(output_name):
#     os.remove(output_name)
#     print(f"기존 파일 삭제")

# gdown.download(f"https://drive.google.com/uc?id={zip_file_id}", output_name, quiet=False)
# print(f"zip 다운")

# # 압축해제 절대경로 설정 했지만 안되는 거 같아서 그냥 두기로
# extract_path = r"bread"

# # 기존 폴더 있으면 삭제
# if os.path.exists(extract_path):
#     shutil.rmtree(extract_path)
#     print(f"기존 폴더 삭제")

# os.makedirs(extract_path, exist_ok=True)

# # 압해
# zip_file_path = os.path.join(os.getcwd(), output_name)

# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# print(f"압축 해제 데이터 경로: {extract_path}")

## 윤곽선/경계선 : Edge 및 Contour Detection 함수

In [ ]:
import os
import cv2

def detect_edges_and_contours(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지를 읽을 수 없어요: {image_path}")
        return None, None

    # 그레이스케일 변환 (경계선 강화!)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Canny Edge Detection
    edges = cv2.Canny(gray, 100, 200)

    # Contour Detection
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_image = image.copy()
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

    return edges, contour_image

dataset_path = "/content/bread/train/images/"
output_path = "/content/bread/edge/"

os.makedirs(output_path, exist_ok=True)

for image_file in os.listdir(dataset_path):
    if image_file.lower().endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(dataset_path, image_file)

        # 경계선 및 윤곽선 추출
        edges, contour_image = detect_edges_and_contours(image_path)
        if edges is not None:

            edge_output = os.path.join(output_path, f"{os.path.splitext(image_file)[0]}_edges.jpg")
            contour_output = os.path.join(output_path, f"{os.path.splitext(image_file)[0]}_contours.jpg")

            cv2.imwrite(edge_output, edges)  # 경계선 이미지 저장
            cv2.imwrite(contour_output, contour_image)  # 윤곽선 이미지 저장

print(f"결과 저장 완료: {output_path}")



결과 저장 완료: /content/bread/edge/


In [ ]:
# shutil.make_archive("/content/bread/edge", 'zip', "/content/bread/edge")

# files.download("/content/bread/edge.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 베이글로 테스트

In [ ]:
# bread = "bagel" # 샘플
# output_path = os.path.join(dataset_path, bread, "edge_contours")
# if os.path.exists(output_path):
#     edge_files = sorted([f for f in os.listdir(output_path) if "edges" in f])[:4]

#     plt.figure(figsize=(10, 5))
#     for i, edge_file in enumerate(edge_files):
#         edge_image = cv2.imread(os.path.join(output_path, edge_file), cv2.IMREAD_GRAYSCALE)
#         plt.subplot(1, 4, i + 1)
#         plt.imshow(edge_image, cmap='gray')
#         plt.title(edge_file)
#     plt.tight_layout()
#     plt.show()
# else:
#     print("_")

### Distance Transform 활용 (임시 중단)

In [ ]:
# def apply_distance_transform(image_path):
#     image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
#     if image is None:
#         return None

#     _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

#     dist_transform = cv2.distanceTransform(binary, cv2.DIST_L2, 3)
#     return dist_transform

# dataset_path = "/content/bread/original_image"

# for bread in os.listdir(dataset_path):
#     bread_images_path = os.path.join(dataset_path, bread, "images")
#     if os.path.exists(bread_images_path):
#         print(f"{bread}...")

#         output_path = os.path.join(dataset_path, bread, "distance_transform")
#         os.makedirs(output_path, exist_ok=True)

#         for image_file in os.listdir(bread_images_path):
#             if image_file.lower().endswith(('.jpg', '.png', '.jpeg')):
#                 image_path = os.path.join(bread_images_path, image_file)

#                 distance_image = apply_distance_transform(image_path)
#                 if distance_image is not None:
#                     output_file = os.path.join(output_path, f"{os.path.splitext(image_file)[0]}_dist.jpg")
#                     cv2.imwrite(output_file, (distance_image * 255).astype("uint8"))  # 0~255로 스케일 조정

#         print(f"{bread}:{output_path} 저장")
#     else:
#         print(f"{bread}: 이미지 없음")


In [ ]:
# bread = "bagel"
# output_path = os.path.join(dataset_path, bread, "distance_transform")

# if os.path.exists(output_path):
#     dist_files = sorted(os.listdir(output_path))[:4]

#     plt.figure(figsize=(10, 5))
#     for i, dist_file in enumerate(dist_files):
#         dist_image = cv2.imread(os.path.join(output_path, dist_file), cv2.IMREAD_GRAYSCALE)
#         plt.subplot(1, 4, i + 1)
#         plt.imshow(dist_image, cmap='jet')
#         plt.title(dist_file)
#     plt.tight_layout()
#     plt.show()
# else:
#     print("_")


## Gabor (텍스처. 일단 중단)

In [ ]:
# def apply_gabor_filter(image):
#     filters = []
#     ksize = 31
#     for theta in np.arange(0, np.pi, np.pi / 8):
#         for sigma in [1, 3, 5]:
#             kern = cv2.getGaborKernel((ksize, ksize), sigma, theta, 10, 0.5, 0, ktype=cv2.CV_32F)
#             filters.append(kern)

#     filtered_images = [cv2.filter2D(image, cv2.CV_8UC3, kern) for kern in filters]
#     return filtered_images

# dataset_path = "/content/bread/"

# for bread in os.listdir(dataset_path):
#     bread_path = os.path.join(dataset_path, bread, "images")

#     if os.path.exists(bread_path):
#         print(f"{bread}🥖")

#         output_path = os.path.join(dataset_path, bread, "gabor_images")
#         os.makedirs(output_path, exist_ok=True)

#         for image_file in os.listdir(bread_path):
#             if image_file.lower().endswith(('.jpg', '.png', '.jpeg')):
#                 image_path = os.path.join(bread_path, image_file)
#                 image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

#                 if image is None:
#                     continue

#                 # gabor_images = apply_gabor_filter(image

#                 for i, gabor_image in enumerate(gabor_images[:8]):
#                     output_file = os.path.join(output_path, f"{os.path.splitext(image_file)[0]}_gabor_{i}.jpg")
#                     cv2.imwrite(output_file, gabor_image)

#         print(f"{bread}: 저장 완료 {output_path}")
#     else:
#         print(f"{bread}: 이미지 없음")

### Gabor 일부 시각화

In [ ]:
# for bread in os.listdir(dataset_path):
#     output_path = os.path.join(dataset_path, bread, "gabor_images")
#     if os.path.exists(output_path):
#         gabor_files = sorted(os.listdir(output_path))[:8]
#         print(f"{bread}🥖")
#         plt.figure(figsize=(12, 8))
#         for i, gabor_file in enumerate(gabor_files):
#             gabor_image = cv2.imread(os.path.join(output_path, gabor_file), cv2.IMREAD_GRAYSCALE)
#             plt.subplot(2, 4, i + 1)
#             plt.imshow(gabor_image, cmap='gray')
#             plt.title(gabor_file)
#         plt.tight_layout()
#         plt.show()


## 변경한 이미지 학습 데이터에 포함하기 (임시 중단)

In [ ]:

# # 원본 이미지와 라벨 경로
# images_path = "/content/bread/train/images"
# labels_path = "/content/bread/train/labels"

# # 이미지 파일 처리
# for image_file in os.listdir(images_path):
#     if "_edges" in image_file or "_contours" in image_file:  # edges와 contours 이미지만 처리
#         # 원본 이미지 이름 추출
#         original_name = image_file.replace("_edges", "").replace("_contours", "")
#         original_label = os.path.join(labels_path, f"{os.path.splitext(original_name)[0]}.txt")

#         # 새 라벨 이름 설정
#         new_label = os.path.join(labels_path, f"{os.path.splitext(image_file)[0]}.txt")

#         # 라벨 복사
#         if os.path.exists(original_label):
#             shutil.copy(original_label, new_label)
#             print(f"라벨 복사 완료: {original_label} → {new_label}")
#         else:
#             print(f"라벨 없음: {original_label}")

## train

In [ ]:
!pip install ultralytics
import torch
from ultralytics import YOLO


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.7/914.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
# YOLOv8 모델 초기화
model = YOLO('yolov8n.pt')

# 학습 시작
model.train(data='/content/bread/data.yaml', epochs=50, imgsz=640, patience=3)

100%|██████████| 6.25M/6.25M [00:00<00:00, 140MB/s]


Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/bread/data.yaml, epochs=50, time=None, patience=3, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tru

100%|██████████| 755k/755k [00:00<00:00, 84.4MB/s]


Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 213MB/s]
